# ゲームプログラミング入門：Pygame

迷路ゲーム

- tick
- draw
- sprite
- maze

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# tick.py
# Pygame
# ライブラリ
import pygame

WIDTH = 640
HEIGHT = 480
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

pygame.init() # Pygame初期化
screen = pygame.display.set_mode((WIDTH, HEIGHT)) # 画面のオブジェクト作成
myfont = pygame.font.Font(None, 32)
myclock = pygame.time.Clock()
loopcnt = 0
endflag = 0

# tick test
while endflag == 0:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            endflag = 1 # ウインドウを閉じたときの処理
    screen.fill(BLACK)
    loopcnt += 1
    imagetext = myfont.render(str(loopcnt), True, WHITE) # ループ回数表示
    screen.blit(imagetext, (100, 50))
    myclock.tick(60) # ループ周期：1/60秒に設定
    pygame.display.flip() # 画面更新

pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# draw.py
# Pygame
# ライブラリ
import pygame

WIDTH = 640
HEIGHT = 480
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

pygame.init() # Pygame初期化
screen = pygame.display.set_mode((WIDTH, HEIGHT)) # 画面のオブジェクト作成
myfont = pygame.font.Font(None, 32)
myclock = pygame.time.Clock()
screen.fill(BLACK)

# draw text
def drawtext(x, y, text):
    imagetext = myfont.render(text, True, WHITE)
    screen.blit(imagetext, (x, y))

# line
drawtext(20, 20, "pygame.draw.line")
startpos = (250, 20)
endpos = (450, 100)
pygame.draw.line(screen, WHITE, startpos, endpos)

# rect
drawtext(20, 130, "pygame.draw.rect")
rect = (250, 130, 200, 100)
pygame.draw.rect(screen, RED, rect)

# circle
drawtext(20, 240, "pygame.draw.circle")
pos = (300, 290)
radius = 50
pygame.draw.circle(screen, GREEN, pos, radius)

# polygon
drawtext(20, 350, "pygame.draw.polygon")
poslist = [(250, 450), (300, 350), (350, 450)]
pygame.draw.polygon(screen, BLUE, poslist)

endflag = 0
while endflag == 0:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            endflag = 1 # ウインドウを閉じたときの処理
    myclock.tick(60) # ループ周期：1/60秒に設定
    pygame.display.flip() # 画面更新

pygame.quit()

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# sprite.py
# Pygame
# ライブラリ
import pygame
import random

WIDTH = 640
HEIGHT = 480
BLUE = (0, 0, 255)

pygame.init() # Pygame初期化
screen = pygame.display.set_mode((WIDTH, HEIGHT)) # 画面のオブジェクト作成
myclock = pygame.time.Clock()

# sprite class
class Spclass(pygame.sprite.Sprite):
    def __init__(self): # 初期化関数
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load("man.png").convert()
        colorkey = self.image.get_at((0, 0))
        self.image.set_colorkey(colorkey) # 透明色の設定
        self.rect = self.image.get_rect()
        self.rect.centerx = random.randrange(WIDTH)
        self.rect.centery = random.randrange(HEIGHT)
        self.x1 = random.randrange(-3, 3)
        self.y1 = random.randrange(-3, 3)
    
    def update(self): # スプライトの移動処理
        self.rect.centerx += self.x1
        self.rect.centery += self.y1
        if self.rect.centerx >= WIDTH or self.rect.centerx < 0:
            self.x1 = - self.x1
        if self.rect.centery >= HEIGHT or self.rect.centery < 0:
            self.y1 = - self.y1

allgroup = pygame.sprite.Group()
for i in range(100):
    allgroup.add(Spclass())

endflag = 0
while endflag == 0:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            endflag = 1
    
    screen.fill(BLUE)
    allgroup.update() # スプライトの移動処理
    allgroup.draw(screen) # スプライトの描画
    myclock.tick(60)
    pygame.display.flip()

pygame.quit()

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# maze.py
# Pygame
# ライブラリ
import pygame
import time

#WIDTH = 640
WIDTH = 800
HEIGHT = 480
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 128, 0)
NAVY = (0, 0, 128)

pygame.init() # Pygame初期化
screen = pygame.display.set_mode((WIDTH, HEIGHT)) # 画面のオブジェクト作成
myfont = pygame.font.Font(None, 48) # Goal!表示
mytime = pygame.font.Font(None, 24) # 経過時間表示
myclock = pygame.time.Clock()
offset = 100
bgx = 0 # BG offset
bgy = 0
size = 32 # sprite size
goal_flag = 0 # ゴールフラグ

# sprite class
class Spclass(pygame.sprite.Sprite):
    def __init__(self, x, y, filename): # 初期化関数
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename).convert()
        colorkey = self.image.get_at((0, 0))
        self.image.set_colorkey(colorkey) # 透明色の設定
        self.x = x
        self.y = y
        self.dir = 0
        self.walking = 0
        self.rect = self.image.get_rect()

# player
class Player(Spclass):
    def update(self): # スプライトの移動処理
        global bgx, bgy, goal_flag
        x1 = [0, 1, 0, -1]
        y1 = [-1, 0, 1, 0]
        key = ["K_UP", "K_RIGHT", "K_DOWN", "K_LEFT"]
        if self.walking == 0:
            newdir = -1
            press = pygame.key.get_pressed()
            if press[pygame.K_UP]:
                newdir = 0
            if press[pygame.K_RIGHT]:
                newdir = 1
            if press[pygame.K_DOWN]:
                newdir = 2
            if press[pygame.K_LEFT]:
                newdir = 3
            if newdir != -1:
                #print(key[newdir]) # デバッグ
                newx = int(self.x/size) + x1[newdir]
                newy = int(self.y/size) + y1[newdir]
                if bgdata[newy][newx] == " ":
                    self.dir = newdir
                    self.walking = 1
        else:
            self.x += x1[self.dir] * 4
            self.y += y1[self.dir] * 4
            #print(self.dir, self.x / size, self.y / size) # デバッグ
            if (self.x % size) == 0 and (self.y % size) == 0:
                self.walking = 0
        
        # 画面のスクロール処理
        if self.x - bgx < offset:
            bgx -= 1
        if self.x - bgx >= WIDTH - offset:
            bgx += 1
        if self.y - bgy < offset:
            bgy -= 1
        if self.y - bgy >= HEIGHT - offset:
            bgy += 1
        self.rect.left = self.x - bgx
        self.rect.top = self.y - bgy
        
        # 衝突判定
        hitlist = pygame.sprite.spritecollide(self, allgroup, False)
        if len(hitlist) >= 2: # 2以上なら宝箱と接触
            goal_flag = 1 # ゴール
            imagetext = myfont.render("GOAL!", True, WHITE)
            screen.blit(imagetext, (260, 150))

# box
class Box(Spclass):
    def update(self):
        global bgx, bgy
        self.rect.left = self.x - bgx
        self.rect.top = self.y - bgy

# 迷路データ
bgdata = [ 
    "11111111111111111111", 
    "1 1     1   1      1", 
    "1   111 1 1 1111 1 1", 
    "11111     1 1    1 1", 
    "1   11111 1 1 1111 1", 
    "1 1   1   1 1    1 1", 
    "111 1   1 1   11 1 1", 
    "1   11111 111 11 1 1", 
    "1 1  1      1 1  1 1", 
    "11111111111111111111"]

blockimage = pygame.image.load("block.png").convert()
bgimage = pygame.Surface((size * 20, size * 10))
bgimage.fill(NAVY)

# 迷路を描画する
for y in range(10):
    for x in range(20):
        if (bgdata[y][x] == " "):
            continue
        bgimage.blit(blockimage, (size * x, size * y))

allgroup = pygame.sprite.Group()
box = Box(size * 18, size * 8, "box.png")
#box = Box(size * 3, size * 2, "box.png")
allgroup.add(box)
player = Player(size * 1, size * 1, "man.png")
allgroup.add(player)

# main loop
endflag = 0
start = time.time() # 開始時刻
end = time.time() # 終了時刻
while endflag == 0:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            endflag = 1
    press = pygame.key.get_pressed()
    if press[pygame.K_ESCAPE]: # ESCキー（終了）
        endflag = 1
    
    screen.fill(BLACK)
    # 迷路の表示
    screen.blit(bgimage, (-bgx, -bgy))
    allgroup.update() # 移動処理
    allgroup.draw(screen) # 描画
    
    # 経過時間表示
    if goal_flag == 0:
        end = time.time()   # 終了時刻
    elapsed_time = end - start # 経過時間
    imagetext = mytime.render(str(int(elapsed_time)) + " seconds", True, WHITE) # ループ回数表示
    screen.blit(imagetext, (50, 50))
    
    myclock.tick(60)
    pygame.display.flip() # 画面更新

pygame.quit()